In [1]:
#IBM Data Science Capstone
#Dec. 2020

# This notebook will be used for the battle of the nieghborhood capstone for the IBM DS Certificate through Coursera

In [2]:
import numpy as np
import pandas as pd

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
